In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
# 安裝相關套件
!pip install numpy
!pip install panda
!pip install  tensorflow tensorflow-datasets matplotlib
import numpy as np
import tensorflow_datasets as tfds

import tensorflow as tf

!pip install  tensorflow-hub
!pip install  tfds-nightly
from tensorflow import keras

import tensorflow_hub as hub
from tensorflow_datasets.core.utils import gcs_utils
gcs_utils.gcs_dataset_info_files = lambda *args, **kwargs: None
gcs_utils.is_dataset_on_gcs = lambda *args, **kwargs: False
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Created wheel for panda: filename=panda-0.3.1-py3-none-any.whl size=7256 sha256=723c3eeccde2052147f44c6e0cc1abbcd7a51d57142e4488b8aa27cf56a35144
  Stored in directory: /home/jovyan/.cache/pip/wheels/e6/3d/81/a3665ce657d35359ca337b1db2975dbe5cd281a88b8982f6b6
Successfully built panda
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 421.8 MB 29 kB/s  eta 0:00:013   |▉                               | 11.4 MB 4.7 MB/s eta 0:01:28     |█▏               

     |████████████████████████████████| 88 kB 1.6 MB/s eta 0:00:011
     |████████████████████████████████| 76 kB 2.9 MB/s  eta 0:00:01
     |████████████████████████████████| 155 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 5.7 MB/s  eta 0:00:01
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=b2ace2eb9a044cd04e2d1165ef350cc0bfdb9d94f09a32806195412627468ebd
  Stored in directory: /home/jovyan/.cache/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
  Created wheel for opt-einsum: filename=opt_einsum-3.1.0-py3-none-any.whl size=61681 sha256=779055f0b559bbacd0b224534d1ddaa3a5562c9630fa37d0042ce2563a488201
  Stored in directory: /home/jovyan/.cache/pip/wheels/21/e3/31/0d3919995e859eff01713d381aac3b6b43c69915a2942e5c65
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7540 sha256=eb911c0d9bf742d55b4658d4fd90663c5fa19d014dc9dc2fd1dc7d9a2909de9e
  Stored in directory: /home/jovyan/.cache

Version:  2.1.0
Eager mode:  True
Hub version:  0.7.0
GPU is NOT AVAILABLE


In [4]:

# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /home/jovyan/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete399ZL5/imdb_reviews-train.tfrecord


Shuffling and writing examples to /home/jovyan/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete399ZL5/imdb_reviews-test.tfrecord


Shuffling and writing examples to /home/jovyan/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete399ZL5/imdb_reviews-unsupervised.tfrecord



Dataset imdb_reviews downloaded and prepared to /home/jovyan/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [5]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [6]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 4s 144ms/step - loss: 1.4223 - accuracy: 0.5061 - val_loss: 0.8507 - val_accuracy: 0.4956
Epoch 2/20
30/30 [==============================] - 5s 177ms/step - loss: 0.7915 - accuracy: 0.5232 - val_loss: 0.7395 - val_accuracy: 0.5524
Epoch 3/20
30/30 [==============================] - 5s 173ms/step - loss: 0.7047 - accuracy: 0.5710 - val_loss: 0.6772 - val_accuracy: 0.5866
Epoch 4/20
30/30 [==============================] - 5s 180ms/step - loss: 0.6471 - accuracy: 0.6098 - val_loss: 0.6309 - val_accuracy: 0.6207
Epoch 5/20
30/30 [==============================] - 5s 153ms/step - loss: 0.6042 - accuracy: 0.6465 - val_loss: 0.5950 - val_accuracy: 0.6540
Epoch 6/20
30/30 [==============================] - 5s 158ms/step - loss: 0.5685 - accuracy: 0.6836 - val_loss: 0.5651 - val_accuracy: 0.6875
Epoch 7/20
30/30 [==============================] - 5s 167ms/step - loss: 0.5360 - accuracy: 0.7146 - val_loss: 0.5373 - val_accuracy: 0.7118
Epoch 

In [10]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.330
accuracy: 0.850


In [11]:
model.metrics_names

['loss', 'accuracy']

In [12]:
results

[0.33037536180749233, 0.84988]

In [3]:
# Text classification with preprocessed text: Movie reviews
(train_data, test_data), info = tfds.load(
    # Use the version pre-encoded with an ~8k vocabulary.
    'imdb_reviews/subwords8k', 
    # Return the train/test datasets as a tuple.
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    # Return (example, label) pairs from the dataset (instead of a dictionary).
    as_supervised=True,
    # Also return the `info` structure. 
    with_info=True) 

Shuffling and writing examples to /home/jovyan/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteSLDEUJ/imdb_reviews-train.tfrecord


Shuffling and writing examples to /home/jovyan/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteSLDEUJ/imdb_reviews-test.tfrecord


Shuffling and writing examples to /home/jovyan/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteSLDEUJ/imdb_reviews-unsupervised.tfrecord


TypeError: in converted code:


    TypeError: tf___get_dataset_from_filename() got an unexpected keyword argument 'do_skip'


In [ ]:
tfds.Split.TRAIN